In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from PIL import Image
from pathlib import Path
from typing import Sequence
from matplotlib.figure import Figure
from matplotlib.gridspec import GridSpec
from matplotlib.axes import Axes
from random import randint
from IPython.display import display

from ba_dev.dataset import MammaliaDataSequence, MammaliaDataImage
from ba_dev.transform import ImagePipeline, BatchImagePipeline
from ba_dev.eval_helpers import set_custom_plot_style, plot_image_with_bbox, draw_bbox_on_ax
from ba_dev.utils import load_path_yaml

# paths = load_path_yaml('/cfs/earth/scratch/kraftjul/BA/data/path_config.yml')
paths = load_path_yaml('/Volumes/ExSSD/UserData/GoogleDrive/01_files/001_Studium/01_Semester/06_FS25/09_BA/data/path_config.yml')


In [2]:
set_custom_plot_style()
cm = 1 / 2.54
scaling_factor = 0.5

export = False

In [ ]:
image_pipeline = ImagePipeline(
                pre_ops=[
                    ('reduce_resolution', {'factor': scaling_factor})
                ]
                )

dataset_img = MammaliaDataImage(
    path_labelfiles=paths['labels'],
    path_to_dataset=paths['dataset'],
    path_to_detector_output=paths['md_output'],
    detector_model=None,
    mode='eval',
    image_pipeline=image_pipeline,
)

image_pipeline = BatchImagePipeline(
                pre_ops=[
                    ('reduce_resolution', {'factor': scaling_factor})
                ]
                )

dataset_seq = MammaliaDataSequence(
    path_labelfiles=paths['labels'],
    path_to_dataset=paths['dataset'],
    path_to_detector_output=paths['md_output'],
    detector_model=None,
    mode='eval',
    image_pipeline=image_pipeline,
)


In [ ]:
df = dataset_img.ds.copy()

summary_df = (
    df.groupby('seq_id', group_keys=False)
      .agg(
          n_images=('conf', 'count'),
          min_conf=('conf', 'min'),
          max_conf=('conf', 'max')
      )
      .reset_index()
)
summary_df['conf_range'] = summary_df['max_conf'] - summary_df['min_conf']

summary_df_sorted = summary_df.sort_values(by='conf_range', ascending=False)


In [ ]:
n_plots = 20
fig_width = 24*cm
target_path = Path('/cfs/earth/scratch/kraftjul/BA/output/conf_examples')

for seq_id in summary_df_sorted['seq_id'].tolist()[:n_plots]:

    i = dataset_seq.get_index_by_seq_id(seq_id)
    sequence = dataset_seq[i]

    length = len(sequence['sample'])

    if length >= 6:
        nrows = 3
        ncols = 2
    else:
        raise ValueError("The sequence is too short to be displayed in a 3x2 grid.")

    fig = plt.figure(figsize=(fig_width, fig_width * nrows / ncols * 3/4))
    gs = GridSpec(
        nrows=nrows, ncols=ncols,
        figure=fig,
        )

    for col in range(ncols):
        for row in range(nrows):
            ax = fig.add_subplot(gs[row, col])

            if col == 0:
                idx = row
            else:
                idx = -(nrows - row)

            img = sequence['sample'][idx]
            bbox = sequence['bbox'][idx]
            conf = sequence['conf'][idx]

            draw_bbox_on_ax(ax, img, bbox, conf)

    fig.tight_layout()
    fig.savefig(target_path / f'seq_{seq_id}.pdf', dpi=300)
    plt.close(fig)
    

In [ ]:
seq_id_to_load = 4000575
i = dataset_seq.get_index_by_seq_id(seq_id_to_load)
sequence = dataset_seq[i]

In [ ]:
len(sequence['sample'])

In [ ]:
sequence['sample']

In [ ]:
interessting_sequences = [4011876, 4018688]

i = dataset_seq.get_index_by_seq_id(interessting_sequences[1])
sequence = dataset_seq[i]

for image, bbox, conf in zip(sequence['sample'], sequence['bbox'], sequence['conf']):
    fig = plot_image_with_bbox(image=image, bbox=bbox, conf=conf)
    display(fig)

In [ ]:
i = randint(0, len(dataset_img) - 1)
print(f"Randomly selected index: {i}")
sample = dataset_img[i]
print(f'class_label: {sample["class_label"]}')
print(f'seq_id: {sample["seq_id"]}')

fig = plot_image_with_bbox(image=sample['sample'], bbox=sample['bbox'], conf=sample['conf'])
display(fig)

In [ ]:
interesting_images = [108812, 340523, 319527]

for i in interesting_images:
    sample = dataset_img[i]
    figure = plot_image_with_bbox(image=sample['sample'], bbox=sample['bbox'], conf=sample['conf'])
    display(figure)

In [ ]:
lowest_conf_indices = dataset_img.ds.nsmallest(5, 'conf').index.tolist()

for i in lowest_conf_indices:
    sample = dataset_img[i]
    figure = plot_image_with_bbox(image=sample['x'], bbox=sample['bbox'], conf=sample['conf'])
    display(figure)
    